In [1]:
import pandas as pd
import numpy as np
import openml
from sklearn.preprocessing import LabelEncoder #For encoding categorical variables
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import KFold,GridSearchCV
from sklearn.metrics import f1_score, make_scorer
import time

# Classification for Iris Dataset

In [2]:
iris = openml.datasets.get_dataset("iris")
iris_df, iris_label, categorical_indicator, attribute_names = iris.get_data(
    target=iris.default_target_attribute, dataset_format="dataframe"
)
iris_df["class"]=iris_label
iris_x=iris_df.iloc[:,:4]
iris_df

sepallength  sepalwidth  petallength  petalwidth           class
0            5.1         3.5          1.4         0.2     Iris-setosa
1            4.9         3.0          1.4         0.2     Iris-setosa
2            4.7         3.2          1.3         0.2     Iris-setosa
3            4.6         3.1          1.5         0.2     Iris-setosa
4            5.0         3.6          1.4         0.2     Iris-setosa
..           ...         ...          ...         ...             ...
145          6.7         3.0          5.2         2.3  Iris-virginica
146          6.3         2.5          5.0         1.9  Iris-virginica
147          6.5         3.0          5.2         2.0  Iris-virginica
148          6.2         3.4          5.4         2.3  Iris-virginica
149          5.9         3.0          5.1         1.8  Iris-virginica

[150 rows x 5 columns]

Alcohol  Malic_acid   Ash  Alcalinity_of_ash  Magnesium  Total_phenols  \
0      14.23        1.71  2.43               15.6        127           2.80   
1      13.20        1.78  2.14               11.2        100           2.65   
2      13.16        2.36  2.67               18.6        101           2.80   
3      14.37        1.95  2.50               16.8        113           3.85   
4      13.24        2.59  2.87               21.0        118           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5         95           1.68   
174    13.40        3.91  2.48               23.0        102           1.80   
175    13.27        4.28  2.26               20.0        120           1.59   
176    13.17        2.59  2.37               20.0        120           1.65   
177    14.13        4.10  2.74               24.5         96           2.05   

     Flavanoids  Nonflavanoid_phenols  Proanthocyanins  Color_intensity   Hue  \
0          3.06                  0.28             2.29             5.64  1.04   
1          2.76                  0.26             1.28             4.38  1.05   
2          3.24                  0.30             2.81             5.68  1.03   
3          3.49                  0.24             2.18             7.80  0.86   
4          2.69                  0.39             1.82             4.32  1.04   
..          ...                   ...              ...              ...   ...   
173        0.61                  0.52             1.06             7.70  0.64   
174        0.75                  0.43             1.41             7.30  0.70   
175        0.69                  0.43             1.35            10.20  0.59   
176        0.68                  0.53             1.46             9.30  0.60   
177        0.76                  0.56             1.35             9.20  0.61   

     OD280%2FOD315_of_diluted_wines  Proline class  
0                              3.92     1065     1  
1                              3.40     1050     1  
2                              3.17     1185     1  
3                              3.45     1480     1  
4                              2.93      735     1  
..                              ...      ...   ...  
173                            1.74      740     3  
174                            1.56      750     3  
175                            1.56      835     3  
176                            1.62      840     3  
177                            1.60      560     3  

[178 rows x 14 columns]

In [4]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   sepallength  150 non-null    float64 
 1   sepalwidth   150 non-null    float64 
 2   petallength  150 non-null    float64 
 3   petalwidth   150 non-null    float64 
 4   class        150 non-null    category
dtypes: category(1), float64(4)
memory usage: 5.1 KB


In [5]:
le=LabelEncoder()

iris_y=le.fit_transform(iris_label)
# wine_y=le.fit_transform(wine_label)

In [6]:
scaler = MinMaxScaler()

iris_x_scaled=scaler.fit_transform(iris_x)
# wine_x_scaled=scaler.fit_transform(wine_x)

In [7]:
iris_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [118]:
def clasification(model,parameters,x,y):
    
    classification_df=pd.DataFrame({"Model":[],
                                    "Random State":[],
                                    "Best Parameters":[],
                                    "Best Score":[],
                                    "Execution Time":[],
                                    "f1 score":[]})
    for i in range(1,11):
        start_time = time.time()
        cv_inner = KFold(n_splits=5, shuffle=True, random_state=i)
        score = ['accuracy' ,'f1_macro']
        grid_search = GridSearchCV(model, parameters, cv=cv_inner, scoring=score, refit='accuracy', n_jobs = -1)
        grid_result = grid_search.fit(x,y)
        f1_score = grid_result.cv_results_['mean_test_f1_macro']
        execution_time = time.time() - start_time
        classification_df=classification_df.append({"Model":model,
                                                    "Random State":i,
                                                    "Best Parameters":grid_result.best_params_,
                                                    "Best Score":grid_result.best_score_,
                                                    "Execution Time":execution_time,
                                                    "f1 score":f1_score},ignore_index=True)
       
    if model == KNC:
        classification_df.to_csv("Iris_KNC.csv")
    elif model == LDA:
        classification_df.to_csv("Iris_LDA.csv")
    elif model == GNB:
        classification_df.to_csv("Iris_GNB.csv")


    return classification_df
    
    
    
#         params = grid_result.cv_results_['params']
#         f1_score = grid_result.cv_results_['mean_test_f1_macro']

## K-nearest neighbour classification

In [119]:
KNC = KNeighborsClassifier()

KNC_grid = {"n_neighbors":range(1, 21),
           "weights":['uniform', 'distance'],
           "metric":['manhattan','euclidean', 'minkowski']}

In [120]:
clasification(KNC,KNC_grid,iris_x_scaled,iris_y)

Model  Random State  \
0  KNeighborsClassifier()           1.0   
1  KNeighborsClassifier()           2.0   
2  KNeighborsClassifier()           3.0   
3  KNeighborsClassifier()           4.0   
4  KNeighborsClassifier()           5.0   
5  KNeighborsClassifier()           6.0   
6  KNeighborsClassifier()           7.0   
7  KNeighborsClassifier()           8.0   
8  KNeighborsClassifier()           9.0   
9  KNeighborsClassifier()          10.0   

                                     Best Parameters  Best Score  \
0  {'metric': 'euclidean', 'n_neighbors': 9, 'wei...    0.973333   
1  {'metric': 'euclidean', 'n_neighbors': 6, 'wei...    0.966667   
2  {'metric': 'euclidean', 'n_neighbors': 6, 'wei...    0.966667   
3  {'metric': 'euclidean', 'n_neighbors': 14, 'we...    0.973333   
4  {'metric': 'euclidean', 'n_neighbors': 14, 'we...    0.973333   
5  {'metric': 'euclidean', 'n_neighbors': 5, 'wei...    0.966667   
6  {'metric': 'euclidean', 'n_neighbors': 2, 'wei...    0.966667   
7  {'metric': 'euclidean', 'n_neighbors': 6, 'wei...    0.966667   
8  {'metric': 'manhattan', 'n_neighbors': 18, 'we...    0.966667   
9  {'metric': 'euclidean', 'n_neighbors': 6, 'wei...    0.960000   

   Execution Time                                           f1 score  
0        0.897597  [0.9221037336254728, 0.9221037336254728, 0.928...  
1        0.929517  [0.9401705761969593, 0.9401705761969593, 0.931...  
2        0.835766  [0.9384541910331384, 0.9384541910331384, 0.938...  
3        0.893612  [0.9308426681677456, 0.9308426681677456, 0.942...  
4        1.154910  [0.9471420500151151, 0.9471420500151151, 0.960...  
5        0.994340  [0.9403688419685189, 0.9403688419685189, 0.945...  
6        1.123992  [0.9416315383514199, 0.9416315383514199, 0.955...  
7        1.046203  [0.9409770067505967, 0.9409770067505967, 0.953...  
8        0.909566  [0.9388938790677919, 0.9388938790677919, 0.952...  
9        0.868674  [0.9305542939583861, 0.9305542939583861, 0.941...

## Linear discriminant analysis 

In [123]:
LDA = LinearDiscriminantAnalysis()

LDA_grid = {"n_components":range(1,3),
           "solver":['lsqr','eigen','svd']}

In [124]:
clasification(LDA,LDA_grid,iris_x_scaled,iris_y)

Model  Random State  \
0  LinearDiscriminantAnalysis()           1.0   
1  LinearDiscriminantAnalysis()           2.0   
2  LinearDiscriminantAnalysis()           3.0   
3  LinearDiscriminantAnalysis()           4.0   
4  LinearDiscriminantAnalysis()           5.0   
5  LinearDiscriminantAnalysis()           6.0   
6  LinearDiscriminantAnalysis()           7.0   
7  LinearDiscriminantAnalysis()           8.0   
8  LinearDiscriminantAnalysis()           9.0   
9  LinearDiscriminantAnalysis()          10.0   

                         Best Parameters  Best Score  Execution Time  \
0  {'n_components': 1, 'solver': 'lsqr'}    0.973333        0.091755   
1  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.050864   
2  {'n_components': 1, 'solver': 'lsqr'}    0.973333        0.048870   
3  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.048867   
4  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.049867   
5  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.048871   
6  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.061834   
7  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.051871   
8  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.053857   
9  {'n_components': 1, 'solver': 'lsqr'}    0.980000        0.054849   

                                            f1 score  
0  [0.9695119929800237, 0.9695119929800237, 0.969...  
1  [0.9817150170091347, 0.9817150170091347, 0.981...  
2  [0.9719973954494078, 0.9719973954494078, 0.971...  
3  [0.9726710239651417, 0.9726710239651417, 0.972...  
4  [0.9808612440191388, 0.9808612440191388, 0.980...  
5  [0.9786384289795247, 0.9786384289795247, 0.978...  
6  [0.9805184914545528, 0.9805184914545528, 0.980...  
7  [0.9820687695368002, 0.9820687695368002, 0.982...  
8  [0.9804526079206386, 0.9804526079206386, 0.980...  
9  [0.9800379572118703, 0.9800379572118703, 0.980...

## Naïve Bayes classifier

In [125]:
GNB = GaussianNB()

var_smoothing= np.logspace(0,-9, num=100)
GNB_grid=dict(var_smoothing=var_smoothing)


In [126]:
clasification(GNB,GNB_grid,iris_x_scaled,iris_y)

Model  Random State                         Best Parameters  \
0  GaussianNB()           1.0  {'var_smoothing': 0.04328761281083057}   
1  GaussianNB()           2.0   {'var_smoothing': 0.0657933224657568}   
2  GaussianNB()           3.0  {'var_smoothing': 0.02848035868435802}   
3  GaussianNB()           4.0   {'var_smoothing': 0.2848035868435802}   
4  GaussianNB()           5.0   {'var_smoothing': 0.2848035868435802}   
5  GaussianNB()           6.0  {'var_smoothing': 0.23101297000831597}   
6  GaussianNB()           7.0  {'var_smoothing': 0.15199110829529336}   
7  GaussianNB()           8.0   {'var_smoothing': 0.1873817422860384}   
8  GaussianNB()           9.0   {'var_smoothing': 0.1873817422860384}   
9  GaussianNB()          10.0   {'var_smoothing': 0.1873817422860384}   

   Best Score  Execution Time  \
0    0.953333        0.551525   
1    0.953333        0.518612   
2    0.946667        0.575461   
3    0.953333        0.666219   
4    0.953333        0.489689   
5    0.953333        0.533568   
6    0.953333        0.640290   
7    0.953333        0.550525   
8    0.953333        0.539559   
9    0.946667        0.570470   

                                            f1 score  
0  [0.8878386136280871, 0.8878386136280871, 0.901...  
1  [0.9164969017910194, 0.9308645747861434, 0.930...  
2  [0.9246210170081486, 0.9246210170081486, 0.917...  
3  [0.8940059853001031, 0.8871326821326821, 0.912...  
4  [0.9414967357166846, 0.9414967357166846, 0.941...  
5  [0.9002068302068302, 0.9115728297467427, 0.917...  
6  [0.9429218428281558, 0.9429218428281558, 0.942...  
7  [0.9099783131522262, 0.9175479725949506, 0.931...  
8  [0.9086023773666794, 0.9086023773666794, 0.915...  
9  [0.904059434929, 0.904059434929, 0.91029201242...

## Support vector machine

In [ ]:

c=
kernel={"linear","poly","rbf","sigmoid","precomputed"}
gamma={"scale","auto"}

In [ ]:
# def clasification(model,parameters,x,y):
#     for i in range(1,10):
#         cv_inner = KFold(n_splits=5, shuffle=True, random_state=i)
#         score = ['accuracy' ,'f1_macro']

#         grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
#         grid_result = grid_search.fit(x,y)
#         print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#         params = grid_result.cv_results_['params']
#         f1_score = grid_result.cv_results_['mean_test_f1_macro']

#     knc_df=pd.DataFrame({"Parameters":[],
#                          "f1_score":[]})

#     for x,y in zip(params,f1_score):
#         knc_df=knc_df.append({"Parameters":x,
#                               "f1_score":y},ignore_index = True)
        
#     return knc_df
    

In [ ]:
KNC = KNeighborsClassifier(n_jobs=-1)

n_neighbors = range(1, 21)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
KNC_grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)



In [ ]:
def clasification(model,parameters,x,y):
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    score = ['accuracy' ,'f1_macro']

    grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
    grid_result = grid_search.fit(x,y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

    params = grid_result.cv_results_['params']
    f1_score = grid_result.cv_results_['mean_test_f1_macro']

    knc_df=pd.DataFrame({"Parameters":[],
                         "f1_score":[]})

    for x,y in zip(params,f1_score):
        knc_df=knc_df.append({"Parameters":x,
                              "f1_score":y},ignore_index = True)
        
    return knc_df

In [ ]:
clasification(LDA,LDA_grid,iris_x_scaled,iris_y)

In [ ]:
model = GaussianNB()

priors=range(1,5)
var_smoothing= np.logspace(0,-9, num=100)
parameters=dict(var_smoothing=var_smoothing)

cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
score = ['accuracy' ,'f1_macro']

grid_search = GridSearchCV(model, parameters, scoring=score, refit="accuracy", error_score=0)
grid_result = grid_search.fit(iris_x_scaled,iris_y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

params = grid_result.cv_results_['params']
f1_score = grid_result.cv_results_['mean_test_f1_macro']

knc_df=pd.DataFrame({"Parameters":[],
                         "f1_score":[]})

for x,y in zip(params,f1_score):
    knc_df=knc_df.append({"Parameters":x,
                          "f1_score":y},ignore_index = True)

knc_df


In [ ]:
wine = openml.datasets.get_dataset("wine")
wine_df, wine_label, categorical_indicator, attribute_names = wine.get_data(
    target= wine.default_target_attribute, dataset_format="dataframe"
)
wine_df["class"]=wine_label
wine_x=wine_df.iloc[:,:13]
wine_df